In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import max,countDistinct,col,regexp_replace

In [2]:
spark = SparkSession.builder.master('local').appName('Walmart Stock').getOrCreate()

In [3]:
df_order_input = spark.read.options(header ='True', InferSchema='True').csv('sales.csv')
df_order_input = df_order_input.withColumnRenamed("product_id","ordered_product_id")
df_seller_input = spark.read.options(header ='True', InferSchema='True').csv('sellers.csv')
df_product_input = spark.read.options(header ='True', InferSchema='True').csv('products.csv')
#df_order_input.show(5)
#df_product_input.show(5)
#df_seller_input.show(5)

+---------+-----------+------------+
|seller_id|seller_name|daily_target|
+---------+-----------+------------+
|        0|   seller_0|     2500000|
|        1|   seller_1|     1375559|
|        2|   seller_2|      205349|
|        3|   seller_3|       71546|
|        4|   seller_4|     1315668|
+---------+-----------+------------+
only showing top 5 rows



In [4]:
##Find out how many orders, how many products and how many sellers are in the data.
print ('Order:' +str(df_order_input.count()),'Product:'+str(df_product_input.count()),'Seller:'+str(df_seller_input.count()))

Order:20000040 Product:75000000 Seller:10


In [5]:
##How many products have been sold at least once? Which is the product contained in more orders?
df_order_input.createOrReplaceTempView("order")
df_multiple_order= spark.sql("select ordered_product_id from(select ordered_product_id,count(*) as count from order group by ordered_product_id having count(*) > 1 )")
df_max_order= spark.sql("select max(count) as max_product_ordered from (select ordered_product_id,count(*) as count from order group by ordered_product_id having count(*) > 1 )")
#df_multiple_order.show()
#df_max_order.show()

In [6]:
##how many distinct products have been sold in each date
df_selected_col = df_order_input.select('date','ordered_product_id').distinct()
df_each_day = df_selected_col.groupBy('date').count().show()

+----------+------+
|      date| count|
+----------+------+
|2020-07-03|100224|
|2020-07-07| 99453|
|2020-07-01| 99755|
|2020-07-08|100048|
|2020-07-04|100294|
|2020-07-10|100218|
|2020-07-09| 99801|
|2020-07-06| 99869|
|2020-07-02| 99768|
|2020-07-05| 99991|
+----------+------+



In [7]:
##What is the average revenue of the orders?
df_order_product = df_order_input.join(df_product_input, df_order_input.ordered_product_id == df_product_input.product_id, 'inner').selectExpr('date','product_id','num_pieces_sold','price')
df_revenue = df_order_product.withColumn('revenue', col("price") * col("num_pieces_sold"))
df_revenue = df_revenue.agg({"revenue": "avg"}).show()

+------------------+
|      avg(revenue)|
+------------------+
|1245.9236386027228|
+------------------+



In [8]:
##“Who are the second most selling and the least selling persons (sellers) for each product? 
##Who are those for the product with product_id = 0”.

In [9]:
##If a product has been sold by only one seller
df_selected_col = df_order_input.select('order_id','seller_id','ordered_product_id','num_pieces_sold')
df_selected_col.createOrReplaceTempView("order")
df_single_seller= spark.sql("select num_pieces_sold,ordered_product_id,count(distinct seller_id) as count FROM order group by ordered_product_id,num_pieces_sold having count = 1 ")

In [10]:
##If a product has been sold by more than one seller, but all of them sold the same quantity
df_selected_col = df_order_input.select('order_id','seller_id','ordered_product_id','num_pieces_sold')
df_selected_col.createOrReplaceTempView("order")
df_multiple_seller= spark.sql("select num_pieces_sold,ordered_product_id,count(distinct seller_id) as count FROM order group by ordered_product_id,num_pieces_sold having count > 1 ")

In [11]:
df_order_input.filter('ordered_product_id == 24185211').show()

+--------+------------------+---------+----------+---------------+--------------------+
|order_id|ordered_product_id|seller_id|      date|num_pieces_sold|       bill_raw_text|
+--------+------------------+---------+----------+---------------+--------------------+
|  482779|          24185211|        3|2020-07-02|             37|yisdoasufImlvwzxv...|
+--------+------------------+---------+----------+---------------+--------------------+



In [12]:
## merging both scenario and creating one df
df_merged = df_single_seller.union(df_multiple_seller)
df_merged = df_merged.withColumnRenamed("ordered_product_id","prod_id")
df_merged = df_merged.withColumnRenamed("num_pieces_sold","num_piece_sold")

In [13]:
cond = [df_merged.prod_id == df_order_input.ordered_product_id, df_merged.num_piece_sold == df_order_input.num_pieces_sold]
df_combined = df_order_input.join(df_merged, cond , 'inner').orderBy('seller_id').selectExpr('order_id','prod_id','seller_id','num_piece_sold')

In [14]:
## take the sum of pieces sold based on seller_id 
df_combined.createOrReplaceTempView("order_output")
df= spark.sql("select prod_id,seller_id,sum(num_piece_sold) as count from order_output where prod_id ='0' group by seller_id,prod_id order by seller_id,prod_id asc ")
#df.show(10)

In [15]:
## using rank function to find out least and second most seller_id and merging both df
df.createOrReplaceTempView("masterorder")
df_second_most_seller= spark.sql("select prod_id,seller_id,count,rn from "+ " (select   prod_id,seller_id,count, RANK() OVER (PARTITION BY prod_id ORDER BY count DESC) as rn " +" FROM masterorder) tmp where rn =2 ")
df_least_most_seller= spark.sql("select prod_id,seller_id,count,rn from "+ " (select   prod_id,seller_id,count, RANK() OVER (PARTITION BY prod_id ORDER BY count ASC) as rn " +" FROM masterorder) tmp where rn =1  ")
df_final_output = df_second_most_seller.union(df_least_most_seller)
#df_final_output.show()

In [17]:
## finding out the seller name
df_final_opt = df_final_output.join(df_seller_input, df_final_output.seller_id == df_seller_input.seller_id, 'inner')
df_final_opt.withColumn('rn', regexp_replace('rn', '2', 'Second-Most Seller')).withColumn('rn', regexp_replace('rn', '1', 'Least-Most Seller')).selectExpr('prod_id','rn','seller_name').show()

+-------+-----------------+-----------+
|prod_id|               rn|seller_name|
+-------+-----------------+-----------+
|      0|Least-Most Seller|   seller_0|
+-------+-----------------+-----------+

